## This will be our initial attempt to develop a script to scrape the website of choice.

In [28]:
import requests 
from bs4 import BeautifulSoup 
    
def getdata(url): 
    r = requests.get(url) 
    return r.text 
        
# htmldata = getdata("https://en.wikipedia.org/wiki/Cheese")
htmldata = getdata("https://www.csulb.edu/college-of-education")
soup = BeautifulSoup(htmldata, 'html.parser')

tgt_img_urls = []
    
for item in soup.find_all('img'):
    if item['src'][:8] == 'https://':
        tgt_img_urls.append(item['src'])
    elif item['src'][:2] == '//':
        https_concat = 'https:'+(item['src'])
        tgt_img_urls.append(https_concat)
        print(tgt_img_urls)
    elif item['src'][:1] == '/':
        https_concat = 'https:/'+(item['src'])
        tgt_img_urls.append(https_concat)
    

In [29]:
# credit to Nate who found this and made it work for us!
import os
import requests
import shutil

# create a directory to store what we will download
# user will need to define their own desired destination folder
os.mkdir('C:\\Users\\hanse\\Documents\\GitHub\\Algorithmic-Bias\\Web-Scraping\\image-folder2')
os.chdir('C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder2')

# more robust way of doing stuff above
# def getdata(url):
#     folder_name = input('File Name:- ')
#     if not os.path.exists(folder_name):
#         os.makedirs(folder_name) 
#     os.chdir(f'{folder_name}')
#     r = requests.get(url) 
#     return r.text

urls = ['https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Cheese_platter.jpg/520px-Cheese_platter.jpg'
       ,'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Ricotta_affumicata_della_sila.jpg/440px-Ricotta_affumicata_della_sila.jpg'
       ,'https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Formaggi.JPG/440px-Formaggi.JPG']
for x in tgt_img_urls:
    try:
        filename = x.split("/")[-1]

    # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(x, stream = True)

    # Check if the image was retrieved successfully
        if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw, f)

            print('Image sucessfully Downloaded: ',filename)
        else:
            print('Image Couldn\'t be retreived')
    except:
        print('error')

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


# <font color='red'> stuff above didnt work fully, stuff below we are adapting 

In [26]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each image, download it
        download(img, path)
        
main("https://www.csulb.edu/college-of-education", "C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder")

Extracting images: 100%|███████████████████████████████████████████████████████████████████████| 23/23 [00:00<?, ?it/s]
